In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15766533
paper_name = 'zhao_houry_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/mmc4.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 200 x 9


In [7]:
original_data.head()

,Count,Systematic name,ORF Name,DNTAG in Strain?,Suspect Tag?,Strain In Pool?,Protein Function Description,Average Log2 Ratio,Std Dev Log2 Ratio
0,1,YAL002W,VPS8,membrane-associated hydrophilic protein which ...,N,Y,membrane-associated hydrophilic protein which ...,3.14830,0.715026
1,2,YAL026C,DRS2,"P-type ATPase, potential aminophospholipid tra...",N,Y,"P-type ATPase, potential aminophospholipid tra...",2.46595,0.767564
2,3,YBL007C,SLA1,cytoskeletal protein binding protein,N,Y,cytoskeletal protein binding protein,3.50285,0.007142
3,4,YBL024W,NCL1,tRNA:m5C-methyltransferase,N,Y,tRNA:m5C-methyltransferase,2.39855,0.027789
4,5,YBL027W,RPL19B,ribosomal protein L19B (YL14) (L23B) (rpl5L),N,Y,ribosomal protein L19B (YL14) (L23B) (rpl5L),1.58190,0.000000


In [8]:
original_data['orf'] = original_data['Systematic name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Count, Systematic name, ORF Name, DNTAG in Strain?, Suspect Tag?, Strain In Pool?, Protein Function Description, Average Log2 Ratio, Std Dev Log2 Ratio, orf]
Index: []


In [12]:
# Taking the opposite because the data is log2 untreated/treated
original_data['data'] = -pd.to_numeric(original_data['Average Log2 Ratio'], errors='coerce')

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(200, 1)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [580]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,580
data_type,value
orf,
YAL002W,-3.14830
YAL026C,-2.46595
YBL007C,-3.50285
YBL024W,-2.39855
YBL027W,-1.58190


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,580
,data_type,value
gene_id,orf,
2,YAL002W,-3.14830
24,YAL026C,-2.46595
95,YBL007C,-3.50285
112,YBL024W,-2.39855
115,YBL027W,-1.58190


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id           580          
data_type          value    valuez
gene_id orf                       
2       YAL002W -3.14830 -5.213957
24      YAL026C -2.46595 -4.083905
95      YBL007C -3.50285 -5.801134
112     YBL024W -2.39855 -3.972283
115     YBL027W -1.58190 -2.619814

# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15766533...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  2.02it/s]

Updating the data_modified_on field...
